In [62]:
import os
import json
import re
from itertools import chain, groupby
import pandas as pd
import numpy as np
import random

In [63]:
with open("dataset/T2IS/data_summary.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

# Group by uid then (height, width)
for d in data:
    d['size'] = (d['height'], d['width'])

In [64]:
df = pd.DataFrame(data)

# 按 uid 分组，然后在每个 uid 组内按 (height, width) 分组
grouped = df.groupby(['uid', 'size'])

grouped_list = {
    name: group.to_dict('records')
    for name, group in grouped
}

In [65]:
len(grouped_list.keys()), np.unique([len(x) for x in grouped_list.values()])

(89, array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 20, 22]))

In [76]:
random.shuffle(data)

In [77]:
train_set = pd.DataFrame(data[:int(len(data)*0.5)])
test_set = pd.DataFrame(data[int(len(data)*0.5):])

grouped_train = train_set.groupby(['uid', 'size'])
grouped_test = test_set.groupby(['uid', 'size'])

grouped_train = {
    name: group.to_dict('records')
    for name, group in grouped_train
}
grouped_test = {
    name: group.to_dict('records')
    for name, group in grouped_test
}

In [78]:
len(grouped_train), len(grouped_test)

(80, 79)

In [79]:
[np.unique([len(x) for x in grouped.values()]) for grouped in [grouped_train, grouped_test]]

[array([ 1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 13]),
 array([ 1,  2,  3,  4,  5,  6,  7, 10, 11, 12])]

In [80]:
output_dir = "dataset/T2IS/train_half"
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "train_metadata.jsonl"), "w") as f:
    for group in grouped_train.values():
        for item in group:
            f.write(json.dumps(item) + "\n")

with open(os.path.join(output_dir, "test_metadata.jsonl"), "w") as f:
    for group in grouped_test.values():
        for item in group:
            f.write(json.dumps(item) + "\n")